<a href="https://colab.research.google.com/github/kszysiec/dw_matrix_car/blob/master/Day5_hyper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw_matrix_car/data'

/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw_matrix_car/data


In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp,fmin,tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

In [0]:
df = df[df['price_currency']!='EUR']
df.shape

(106290, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if (isinstance(df[feat][0],list)):continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] =factorized_values
  else:
    df[feat+SUFFIX_CAT] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df["price_value"].values

In [0]:
def run_model(model,feats):
  X = df[feats].values
  y = df["price_value"].values
  scores = cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x:-1 if str(x)=='None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x:-1 if str(x)=='None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x:-1 if str(x)=='None' else int(x.split('cm')[0].replace(' ','')))

In [0]:
feats = ['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_kod-silnika__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat',
'feature_światła-led']

In [0]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate': 0.1,
    'seed':0
}

model = xgb.XGBRegressor(**xgb_params)

In [0]:
run_model(model,feats)

[17:19:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:19:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:19:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9449.513980284812, 81.47168211987172)

In [0]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
  return {'loss':np.abs(mean_mae), 'status':STATUS_OK}

In [0]:
xgb_reg_params = {
    'learning_rate':hp.choice('learning_rate', np.arange(0.05,0.31,0.05)),
    'max_depth':hp.choice('max_depth', np.arange(5,16,1,dtype = int)),
    'subsample':hp.quniform('subsample', 0.5,1,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree', 0.5,1,0.05),
    'objective':'reg:squarederror',
    'n_estimators':100,
    'seed':0
}

In [0]:
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best


Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.05, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.05, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.2, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_

{'colsample_bytree': 0.8,
 'learning_rate': 0,
 'max_depth': 9,
 'subsample': 0.75}